In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os.path

import matplotlib.pyplot as plt
import pandas as pd
from arcgis.gis import GIS
from arcgis.raster import ImageryLayer
from dotenv import load_dotenv
import rioxarray
import xarray as xr
import xvec
# from queries import query_df

# load snowflake connection details
load_dotenv()
print(xvec.__version__)  # do not remove import

/Users/markns/workspace/farmwise/.venv/lib/python3.12/site-packages/arcgis/auth/_auth/_token.py:479: SyntaxWarning: invalid escape sequence '\s'
  "step-1c": re.compile("var\s+(\w+)\s*=\s*({.*?})", re.DOTALL),
/Users/markns/workspace/farmwise/.venv/lib/python3.12/site-packages/arcgis/gis/__init__.py:6064: SyntaxWarning: invalid escape sequence '\s'
  """
/Users/markns/workspace/farmwise/.venv/lib/python3.12/site-packages/arcgis/gis/__init__.py:9672: SyntaxWarning: invalid escape sequence '\{'
  new_path = f"{current_path}\{title}" if current_path else title
/Users/markns/workspace/farmwise/.venv/lib/python3.12/site-packages/arcgis/gis/__init__.py:14383: SyntaxWarning: invalid escape sequence '\A'
  """
/Users/markns/workspace/farmwise/.venv/lib/python3.12/site-packages/arcgis/_impl/common/_deprecate.py:104: SyntaxWarning: invalid escape sequence '\s'
  """Decorate a function or class to signify its deprecation
/Users/markns/workspace/farmwise/.venv/lib/python3.12/site-packages/arcgis/f

0.3.1


In [29]:
# Connect to GIS
gis = GIS()

gaez_image_services = {
    "Land and Water Resources": "https://gaez-services.fao.org/server/rest/services/LR/ImageServer",
    "Agro-Climate Resources": "https://gaez-services.fao.org/server/rest/services/res01/ImageServer",
    "Agro-Climate Yield": "https://gaez-services.fao.org/server/rest/services/res02/ImageServer",
    "Suitability and Attainable Yield": "https://gaez-services.fao.org/server/rest/services/res05/ImageServer",
    "Actual Yield and Production": "https://gaez-services.fao.org/server/rest/services/res06/ImageServer",
    "Yield and Production Gaps": "https://gaez-services.fao.org/server/rest/services/res07/ImageServer"
}



In [12]:

theme = "Land and Water Resources"

# Access the GAEZ ImageServer
imagery_layer = ImageryLayer(gaez_image_services[theme])

# Define the definition query
#
where_clause = "rcp = 'RCP4.5' and model = 'ENSEMBLE' and year = '2011-2040'"

# Define pagination parameters
max_records = 1000  # Set the number of records per page
result_offset = 0  # Initial offset
all_features = []  # To store all results

while True:
    query_result = imagery_layer.query(where=where_clause, out_fields="*",
                                       result_record_count=max_records,
                                       result_offset=result_offset)
    all_features.extend(query_result.features)

    # If the number of features returned is less than max_records, we've reached the end
    if len(query_result.features) < max_records:
        break
    result_offset += max_records

df = pd.DataFrame([feature.attributes for feature in all_features])
df

,OBJECTID,Name,Category,sub_theme_name,variable,file_description,year,model,rcp,units,renderer,download_url,Thumbnail,file_id
0,591,aez_v9v2red_5m_ENSEMBLE_rcp4p5_2020s,1,Agro-ecological Zones,Dominant AEZ class (33 classes) at 5 arc-minutes,Dominant AEZ class (33 classes) at 5 arc-minutes,2011-2040,ENSEMBLE,RCP4.5,Class,AEZ Classification (33 classes),https://s3.eu-west-1.amazonaws.com/data.gaezde...,None,000000025
1,604,aez_v9v2red_ENSEMBLE_rcp4p5_2020s,1,Agro-ecological Zones,Simplified AEZ classification (33 classes),Simplified AEZ classification (33 classes),2011-2040,ENSEMBLE,RCP4.5,Class,AEZ Classification (33 classes),https://s3.eu-west-1.amazonaws.com/data.gaezde...,None,000000034
2,617,aez_v9v2_ENSEMBLE_rcp4p5_2020s,1,Agro-ecological Zones,AEZ classification by climate/soil/terrain/LC ...,AEZ classification by climate/soil/terrain/LC ...,2011-2040,ENSEMBLE,RCP4.5,Class,AEZ Classification (57 classes),https://s3.eu-west-1.amazonaws.com/data.gaezde...,None,000000016
3,630,mst_class_ENSEMBLE_rcp4p5_2020s,1,Agro-ecological Zones,Moisture regime class,Moisture regime class,2011-2040,ENSEMBLE,RCP4.5,Class,Moisture Regime Class,https://s3.eu-west-1.amazonaws.com/data.gaezde...,None,000000047
4,644,thz_class_ENSEMBLE_rcp4p5_2020s,1,Agro-ecological Zones,Thermal regime class,Thermal regime class,2011-2040,ENSEMBLE,RCP4.5,Class,Thermal Regime Class,https://s3.eu-west-1.amazonaws.com/data.gaezde...,None,000000061


In [16]:

extent = {
    "xmin": 33.909,
    "ymin": -4.676,
    "xmax": 41.899,
    "ymax": 5.506,
    "spatialReference": {"wkid": 4326},  # WGS84
}

# Calculate the number of pixels in each dimension
width_pixels = int((extent['xmax'] - extent['xmin']) / imagery_layer.properties['pixelSizeX'])
height_pixels = int((extent['ymax'] - extent['ymin']) / imagery_layer.properties['pixelSizeY'])


def download_gaez_raster(gaez_object, save_folder, file_name):
    os.makedirs(save_folder, exist_ok=True)
    print(f"downloading {gaez_object}")

    # mosaic_rule = f'{{"mosaicMethod":"esriMosaicNorthwest","where":"(((sub_theme_name = \'{gaez_object.sub_theme_name}\') AND (variable = \'{gaez_object.variable}\') AND (year = \'{gaez_object.year}\') AND (model = \'{gaez_object.model}\') AND (rcp = \'{gaez_object.rcp}\') AND (crop = \'{gaez_object.crop}\') AND (water_supply = \'{gaez_object.water_supply}\') AND (input_level = \'{gaez_object.input_level}\') AND (c02_fertilization = \'{gaez_object.c02_fertilization}\')))","sortField":"","ascending":true,"mosaicOperation":"MT_FIRST"}}'
    mosaic_rule = f'{{"mosaicMethod":"esriMosaicNorthwest","where":"(((file_id = \'{gaez_object.file_id}\')))","sortField":"","ascending":true,"mosaicOperation":"MT_FIRST"}}'
    imagery_layer.export_image(bbox=extent,
                               image_sr={"wkid": 4326},
                               bbox_sr={"wkid": 4326},
                               export_format="tiff",
                               size=[width_pixels, height_pixels],  # Full resolution dimensions in pixels
                               f="image",
                               mosaic_rule=mosaic_rule,
                               save_folder=save_folder,
                               save_file=file_name)


for gaez_object in df.itertuples():
    print(gaez_object)
    full_path = gaez_object.download_url.replace('https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/', '')
    if not os.path.exists(full_path):
        save_folder, file_name = os.path.split(full_path)
        download_gaez_raster(gaez_object, save_folder, file_name)


Pandas(Index=0, OBJECTID=591, Name='aez_v9v2red_5m_ENSEMBLE_rcp4p5_2020s', Category=1, sub_theme_name='Agro-ecological Zones', variable='Dominant AEZ class (33 classes) at 5 arc-minutes', file_description='Dominant AEZ class (33 classes) at 5 arc-minutes', year='2011-2040', model='ENSEMBLE', rcp='RCP4.5', units='Class', renderer='AEZ Classification (33 classes)', download_url='https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/LR/aez/aez_v9v2red_5m_ENSEMBLE_rcp4p5_2020s.tif', Thumbnail=None, file_id='000000025')
downloading Pandas(Index=0, OBJECTID=591, Name='aez_v9v2red_5m_ENSEMBLE_rcp4p5_2020s', Category=1, sub_theme_name='Agro-ecological Zones', variable='Dominant AEZ class (33 classes) at 5 arc-minutes', file_description='Dominant AEZ class (33 classes) at 5 arc-minutes', year='2011-2040', model='ENSEMBLE', rcp='RCP4.5', units='Class', renderer='AEZ Classification (33 classes)', download_url='https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/LR/aez/aez_v9v2red_5m

In [18]:
da = rioxarray.open_rasterio("LR/aez/aez_v9v2red_ENSEMBLE_rcp4p5_2020s.tif")
da

<xarray.DataArray (band: 1, y: 1221, x: 958)> Size: 1MB
[1169718 values with dtype=uint8]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 8kB 33.91 33.92 33.93 33.94 ... 41.88 41.89 41.89
  * y            (y) float64 10kB 5.503 5.494 5.486 ... -4.656 -4.664 -4.673
    spatial_ref  int64 8B 0
Attributes:
    DataType:         Generic
    AREA_OR_POINT:    Area
    SourceBandIndex:  0
    scale_factor:     1.0
    add_offset:       0.0

In [28]:
# 0 255 255 255 255 0
# 1 247 159 192 255 Tropics, lowland; semi-arid
# 2 232 79 121 255 Tropics, lowland; sub-humid
# 3 216 28 78 255 Tropics, lowland; humid
def read_clr(filepath):
    colormap = {}
    with open(filepath, 'r') as f:
        for line in f:
            if line.strip() and not line.startswith('#'):
                parts = line.split(maxsplit=5)
                if len(parts) >= 4:
                    value = int(parts[0])
                    label = parts[5].strip()
                    colormap[value] = label
    return colormap

# Usage
clr_path = "GAEZ4_symbology_files/clr_files/AEZ_33classes.clr"
class_map = read_clr(clr_path)
print(class_map)

{0: '0', 1: 'Tropics, lowland; semi-arid', 2: 'Tropics, lowland; sub-humid', 3: 'Tropics, lowland; humid', 4: 'Tropics, highland; semi-arid', 5: 'Tropics, highland; sub-humid', 6: 'Tropics, highland; humid', 7: 'Sub-tropics, warm; semi-arid', 8: 'Sub-tropics, warm; sub-humid', 9: 'Sub-tropics, warm; humid', 10: 'Sub-tropics, moderately cool; semi-arid', 11: 'Sub-tropics, moderately cool; sub-humid', 12: 'Sub-tropics, moderately cool; humid', 13: 'Sub-tropics, cool; semi-arid', 14: 'Sub-tropics, cool; sub-humid', 15: 'Sub-tropics, cool; humid', 16: 'Temperate, moderate; dry', 17: 'Temperate, moderate; moist', 18: 'Temperate, moderate; wet', 19: 'Temperate, cool; dry', 20: 'Temperate, cool; moist', 21: 'Temperate, cool; wet', 22: 'Cold, no permafrost; dry', 23: 'Cold, no permafrost; moist', 24: 'Cold, no permafrost; wet', 25: 'Dominantly very steep terrain', 26: 'Land with severe soil/terrain limitations', 27: 'Land with ample irrigated soils', 28: 'Dominantly hydromorphic soils', 29: 'D

In [21]:
da = da.rio.reproject("EPSG:4326")
# Latitude and longitude (WGS84)
lat = -1.84
lon = 37.62

# Ensure coordinates are named correctly and use nearest match
value = da.sel(x=lon, y=lat, method="nearest").item()

print("Extracted value:", value)


Extracted value: 2


In [32]:
# Theme 2

theme = "Agro-Climate Resources"

# Access the GAEZ ImageServer
imagery_layer = ImageryLayer(gaez_image_services[theme])

# Define the definition query
#
where_clause = "rcp = 'RCP4.5' and model = 'ENSEMBLE' and year = '2011-2040'"

# Define pagination parameters
max_records = 1000  # Set the number of records per page
result_offset = 0  # Initial offset
all_features = []  # To store all results

while True:
    query_result = imagery_layer.query(where=where_clause, out_fields="*",
                                       result_record_count=max_records,
                                       result_offset=result_offset)
    all_features.extend(query_result.features)

    # If the number of features returned is less than max_records, we've reached the end
    if len(query_result.features) < max_records:
        break
    result_offset += max_records

df = pd.DataFrame([feature.attributes for feature in all_features])
df

#
# for gaez_object in df.itertuples():
#     print(gaez_object)
#     full_path = gaez_object.download_url.replace('https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/', '')
#     if not os.path.exists(full_path):
#         save_folder, file_name = os.path.split(full_path)
#         download_gaez_raster(gaez_object, save_folder, file_name)


,objectid,name,category,filepath,sub_theme_name,variable,file_description,year,model,rcp,units,renderer,download_url,file_id
0,226741,et0_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\et0_ENSEMBLE_rcp4p5_202...,Moisture Regime,Reference potential evapotranspiration (using ...,Reference potential evapotranspiration (using ...,2011-2040,ENSEMBLE,RCP4.5,mm,Agro-climatic Resources Symbology - Precipitation,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107761
1,226744,eta_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\eta_ENSEMBLE_rcp4p5_202...,Moisture Regime,Reference actual evapotranspiration (using AWC...,Reference actual evapotranspiration (using AWC...,2011-2040,ENSEMBLE,RCP4.5,mm,Agro-climatic Resources Symbology - Precipitation,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107781
2,226747,fss_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\fss_ENSEMBLE_rcp4p5_202...,Thermal Regime,Snow-adjusted air frost number,Snow-adjusted air frost number for the time pe...,2011-2040,ENSEMBLE,RCP4.5,Index,Thermal Regime Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107698
3,226750,fst_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\fst_ENSEMBLE_rcp4p5_202...,Thermal Regime,Air frost number,Air frost number for the time period 2011-2040...,2011-2040,ENSEMBLE,RCP4.5,Index,Thermal Regime Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107688
4,226753,lac_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\lac_ENSEMBLE_rcp4p5_202...,Growing Period,Adjusted LGP for evaluating AgClim Constraints...,Adjusted LGP for evaluating AgClim Constraints...,2011-2040,ENSEMBLE,RCP4.5,days,0,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107682
5,226756,lb2_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\lb2_ENSEMBLE_rcp4p5_202...,Growing Period,Beginning date of earliest growing period (day...,Beginning date of earliest growing period (day...,2011-2040,ENSEMBLE,RCP4.5,days,0,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107708
6,226759,ld1_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\ld1_ENSEMBLE_rcp4p5_202...,Growing Period,Length of longest component LGP (days),Length of longest component LGP (days) for the...,2011-2040,ENSEMBLE,RCP4.5,days,Agro-climatic Resources Symbology - Days,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107766
7,226762,ld2_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\ld2_ENSEMBLE_rcp4p5_202...,Growing Period,Total number of LGP days in component LGPs > 2...,Total number of LGP days in component LGPs > 2...,2011-2040,ENSEMBLE,RCP4.5,days,0,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107755
8,226765,lgb_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\lgb_ENSEMBLE_rcp4p5_202...,Growing Period,Beginning date of longest component LGP (day-o...,Beginning date of longest component LGP (day-o...,2011-2040,ENSEMBLE,RCP4.5,Number,Beginning Date of Longest Component LGP,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107731
9,226768,lgd_ENSEMBLE_rcp4p5_2020s,1,\res01\ENSEMBLE\rcp4p5\lgd_ENSEMBLE_rcp4p5_202...,Growing Period,Total number of growing period days,Total number of growing period days for the ti...,2011-2040,ENSEMBLE,RCP4.5,days,Agro-climatic Resources Symbology - Days,https://s3.eu-west-1.amazonaws.com/data.gaezd...,000107730


In [36]:
theme = "Suitability and Attainable Yield"

# Access the GAEZ ImageServer
imagery_layer = ImageryLayer(gaez_image_services[theme])

# Define the definition query
#
where_clause = ("rcp = 'RCP4.5' and model = 'ENSEMBLE' and year = '2011-2040' and water_supply = 'Rainfed' "
                "and c02_fertilization = 'Without CO2 Fertilization' and variable = 'Suitability index  range (0 – 10000); current cropland in grid cell'")

# Define pagination parameters
max_records = 1000  # Set the number of records per page
result_offset = 0  # Initial offset
all_features = []  # To store all results

while True:
    query_result = imagery_layer.query(where=where_clause, out_fields="*",
                                       result_record_count=max_records,
                                       result_offset=result_offset)
    all_features.extend(query_result.features)

    # If the number of features returned is less than max_records, we've reached the end
    if len(query_result.features) < max_records:
        break
    result_offset += max_records

df = pd.DataFrame([feature.attributes for feature in all_features])
df



,objectid,name,category,filepath,sub_theme_name,variable,file_description,year,model,rcp,crop,water_supply,input_level,c02_fertilization,units,renderer,download_url,file_id
0,248418,suHr0_alf,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_alf.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Alfalfa,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734670
1,248419,suHr0_ban,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_ban.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Banana,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734671
2,248420,suHr0_brl,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_brl.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Barley,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734672
3,248421,suHr0_cit,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_cit.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Citrus,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734673
4,248422,suHr0_coc,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_coc.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Cocoa,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734674
5,248423,suHr0_cof,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_cof.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Coffee,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734675
6,248424,suHr0_con,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_con.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Coconut,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734676
7,248425,suHr0_cot,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_cot.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Cotton,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734677
8,248426,suHr0_csv,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_csv.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Cassava,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734678
9,248427,suHr0_flx,1,\res05\ENSEMBLE\rcp4p5\2020sH\suHr0_flx.tif,Suitability Index,Suitability index range (0 – 10000); current ...,Suitability index range (0 – 10000); current ...,2011-2040,ENSEMBLE,RCP4.5,Flax,Rainfed,High,Without CO2 Fertilization,Index,Suitability and Attainable Yield Symbology,https://s3.eu-west-1.amazonaws.com/data.gaezde...,000734679


In [52]:

# sdf = df
# df['code'] = df.name.str.split("_")[1]
df[['var','code']] = df.name.str.split("_", expand=True)
variables = pd.Series(df['crop'].values, index=df['code']).to_dict()

file_path = "res05/HadGEM2-ES/rcp4p5/2020sH"


# variables = dict(zip(sdf.name, sdf.crop))
# # variables
rasters = [rioxarray.open_rasterio(os.path.join(file_path, f"suHr0_{v}.tif")) for v in variables.keys()]
# Align rasters to ensure they have the same spatial resolution and extents
rasters = xr.align(*rasters, join="exact")
# Stack rasters along the band dimension
suitability_index = xr.concat(rasters, dim="band")
# Assign meaningful labels to the band dimension
suitability_index = suitability_index.assign_coords(band=list(variables.keys()))
suitability_index

<xarray.DataArray (band: 30, y: 122, x: 95)> Size: 695kB
array([[[  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        ...,
        [5556, 5556, 5556, ...,    0,    0,    0],
        [5556, 5556, 5556, ...,    0,    0,    0],
        [5556, 5556, 5556, ...,    0,    0,    0]],

       [[  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]],

       [[  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        ...,
...
        ...,
        [5556, 5556, 5556, ...,    0,    0,    0],
        [5556, 5556, 5556, ...,    0,    0,    0],
        [5556, 5556, 5556, ...,    0,    0,    0]],

       [[  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        ...,
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0],
        [   0,    0,    0, ...,    0,    0,    0]],

       [[  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        [  -1,   -1,   -1, ...,   -1,   -1,   -1],
        ...,
        [6558, 3333, 3771, ...,    0,    0,    0],
        [3333, 3333, 4357, ...,    0,    0,    0],
        [3333, 3376, 6667, ...,    0,    0,    0]]], dtype=int16)
Coordinates:
  * x            (x) float64 760B 33.95 34.04 34.12 34.2 ... 41.69 41.77 41.86
  * y            (y) float64 976B 5.503 5.419 5.335 ... -4.505 -4.589 -4.673
    spatial_ref  int64 8B 0
  * band         (band) <U3 360B 'alf' 'ban' 'brl' 'cit' ... 'whe' 'wpo' 'yam'
Attributes:
    DataType:         Generic
    AREA_OR_POINT:    Area
    SourceBandIndex:  0
    scale_factor:     1.0
    add_offset:       0.0

In [70]:

lat = -1.84
lon = 37.62

# Ensure coordinates are named correctly and use nearest match
suitability_point = suitability_index.sel(x=lon, y=lat, method="nearest")
dict(zip(suitability_point.band.values, suitability_point.values))

{'alf': 5556,
 'ban': 0,
 'brl': 0,
 'cit': 5556,
 'coc': 0,
 'cof': 0,
 'con': 0,
 'cot': 10000,
 'csv': 7778,
 'flx': 0,
 'grd': 10000,
 'jtr': 5556,
 'mis': 5556,
 'nap': 5556,
 'olp': 0,
 'olv': 0,
 'rcg': 0,
 'rsd': 4848,
 'rub': 0,
 'sfl': 7778,
 'soy': 7778,
 'spo': 7778,
 'sub': 0,
 'suc': 0,
 'swg': 0,
 'tea': 0,
 'tob': 7778,
 'whe': 5556,
 'wpo': 0,
 'yam': 7778}